In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import pandas as pd
from tqdm import tqdm

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from unidecode import unidecode
from umap import UMAP
import hdbscan

import matplotlib.pyplot as plt
import seaborn as sns

from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
import numpy as np



c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../datasets/fakeWhatsApp.BR_2022.csv')
df.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249


In [3]:
ids_para_remover = ['6ef561ec0f448afcd7b3751124bb0712','d3e678a0ba0e1485548260a7c4599152',
                    'ac5703154484de05336af617455ca55e','a39edbd64d378226ffa60433649a0acf',
                    'c66d0d4ae5a4b281bff67e1fa4fbd6ba','819bbc872ed6d81f44d746b710eecf06',
                    'ec94da4d54f9a5693e88fa582926be53','6a38c72316d87c028dfd66c10442476b',
                    '94099e1e46f129856541e2b3640896d1','8f367d1693fff47218603fa47ded525c',
                    'c09caffee0d1bd30926dea9df25dc88f']

In [4]:
# Filtrar o DataFrame para remover as linhas com esses IDs
df_filtrado = df[~df['id_member_anonymous'].isin(ids_para_remover)]
df_filtrado

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,5,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,5,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
2,2022-10-11 00:39:31,c882172d447798d74915973ac83eba68,b84dfe2d1599b82768dcdecce7e6bb23,d2e0ec59ffd9f84764f5b147725d7196.oga,audio/ogg; codecs=opus,NaN,True,False,False,NaN,5,2022-10-11 00:39:33.445125,NaN,NaN,737948BE86D450A426470794F91BC80D,558594228826.0:12@s.whatsapp.net,Audio,whatsapp,NaN,d2e0ec59ffd9f84764f5b147725d7196
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,5,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN
4,2022-10-10 23:40:12,77c1a8a31fee269db258a028a61f0b88,a5831b109d4d95fae8ee0ce464e48a6a,bb80cd530332bb6b95c34e719992d249.f4v,video/mp4,NaN,True,False,False,NaN,5,2022-10-10 23:40:14.397495,NaN,NaN,092203A082AC3DFB2A4933F60453AEB8,558594228826.0:12@s.whatsapp.net,Video,whatsapp,NaN,bb80cd530332bb6b95c34e719992d249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598966,2022-11-13 00:46:56,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,7719ba57cbe74eb5c8059d6b1a14d1ca.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.262572,NaN,NaN,08E682716448AB06DEE126A30E7AAA77,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,7719ba57cbe74eb5c8059d6b1a14d1ca
598967,2022-11-13 13:37:03,2ad743e9ff831a75911a983b42205845,64c9a7c8ac338faa258e5aea9a1c2ae3,c6d59f0b86e63dbcce7fc682a1eaea50.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.269632,NaN,NaN,3EB0259ED9B8FACF18A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,c6d59f0b86e63dbcce7fc682a1eaea50
598968,2022-11-13 17:43:22,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,003ace942d9665adc9647df9d9526c0c.jpeg,image/jpeg,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.277283,NaN,NaN,5CD9F23CD07BEE38693E68E11F9C6A55,558594228826.0:17@s.whatsapp.net,Imagem,whatsapp,NaN,003ace942d9665adc9647df9d9526c0c
598969,2022-11-13 21:40:47,b1b419f54664f6cd4fcdb40425862d36,b1b419f54664f6cd4fcdb40425862d36,bab5653a94b097404739a061dbbc7407.f4v,video/mp4,NaN,True,False,False,NaN,5,2023-01-30 18:20:22.284021,NaN,NaN,D3BDE4A5C68E7148C0925F6C582C99A9,558594228826.0:17@s.whatsapp.net,Video,whatsapp,NaN,bab5653a94b097404739a061dbbc7407


In [5]:
# Removendo linhas com valores NaN na coluna 'text_content_anonymous'
df_filtrado = df_filtrado.dropna(subset=['text_content_anonymous'])

# Removendo trava_zap
df_filtrado = df_filtrado[df_filtrado['trava_zap'] == False]



len(df_filtrado)

254480

In [6]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

custom_stopwords = {
    't', 'be', 'nao', 'youtu', 'vai', 'av', 'ja', 'to', 'the', 'this', 'i', 'and', 
    'you', 'y', 'www', 'sao', 'pois', 'contra', 'user', 'ai', 'so', 'gente', 'voce', 'of', 
    'ola', 'gift', 'card', 'kwaivideo', 'r', 'bom', 'q', 'vc', 'vcs', 'pra','ta', 'phone', 'ok', 'la',
    'sera', 'ha', 'aqui', 'ate', 'dia', 'mc', 'im', 'tmj', 'pix', 'g', 'diz', 'ti', 'etc', 'tudo', 
    'todo', 'toda', 'youtube', 'g1', 'm', 'instagram', 'fb', 'in', 'link', 'was', 'blocked', 'kk'
}

stop_words.update(custom_stopwords)

def preprocess_text(text):

    # Função para extrair e substituir o domínio da URL
    def substituir_dominios(texto):
        # Função para extrair e substituir o domínio da URL
        def extrair_dominio(url):
            # Remove o protocolo (http://, https://, etc.) e o "www." se presente
            dominio = re.sub(r'^https?://(?:www\.)?|www\.', '', url)
            # Remove o caminho e parâmetros da URL
            dominio = re.split(r'[/?#]', dominio)[0]
            # Retorna a parte principal do domínio (antes do primeiro ponto)
            return dominio.split('.')[0]

        # Substitui URLs por seus domínios principais
        return re.sub(r'https?://(?:www\.)?\S+|www\.\S+', lambda match: extrair_dominio(match.group(0)), texto)

    # Substituir domínios
    text = substituir_dominios(text)

    # Converte para minúsculas
    text = text.lower()

    # Remove acentos
    text = unidecode(text)

    #Remover Pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove URLs e menções
    #text = re.sub(r'http\S+|www\S+|https\S+|@\w+', '', text)

    # Substitui emojis repetidos por apenas um
    text = re.sub(r'([\U00010000-\U0010FFFF])\1+', r'\1', text)
    text = re.sub(r'([\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U0001F700-\U0001F77F]|[\U0001F780-\U0001F7FF]|[\U0001F800-\U0001F8FF]|[\U0001F900-\U0001F9FF]|[\U0001FA00-\U0001FA6F]|[\U0001FA70-\U0001FAFF])\1+', r'\1', text)


    # Remove espaços em branco extras (início ou final) e múltiplos espaços no meio do texto
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove pontuações e caracteres especiais
    #text = re.sub(r'[^\w\s]', '', text)

    # Ajusta risadas "kkk" ou mais para "kk"
    text = re.sub(r'k{2,}|K{2,}', 'kk', text)

    # Ajusta risadas "haha" ou mais para "haha"
    text = re.sub(r'(ha){2,}', 'haha', text, flags=re.IGNORECASE)

    # Ajusta risadas "kaka" ou mais para "kaka"
    text = re.sub(r'(ka){2,}', 'kaka', text, flags=re.IGNORECASE)

    # Remove as stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])


    return text


[nltk_data] Downloading package punkt to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Melissa
[nltk_data]     Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:

df_geral = df_filtrado.copy()

# Aplicar o pré-processamento à coluna de texto
df_geral['text_processed'] = df_geral['text_content_anonymous'].apply(preprocess_text)
df_geral.head()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,...,date_system,score_sentiment,score_misinformation,id_message,id_persona,message_type,messenger,media_name,media_md5,text_processed
0,2022-10-10 18:20:24,9d737b3c9387855139bbad2311cc5709,0638569ee76dac58f59dcac20463c955,NaN,NaN,NaN,False,False,False,☝️\nHoje que eu fique sabendo do CANAL LULA FL...,...,2022-10-10 18:20:25.000937,-0.7003,0.843775,F7023FFB06C429A2C166922849A35ED8,558594228826.0:12@s.whatsapp.net,Texto,whatsapp,NaN,NaN,hoje fique sabendo canal lula flix vi pt entro...
1,2022-10-10 22:02:58,1660a60f661754d2802ca53296e25be8,a5910d5cc1c830ade9eb4dd00f15ff6a,NaN,NaN,NaN,False,False,False,Mais pra que isso não aconteça nois temos quê ...,...,2022-10-10 22:02:58.4682,-0.2960,NaN,A9FAC78070C144890D181EF415B90CAD,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,aconteca nois fazer anossa parte
3,2022-10-10 23:36:19,91e3c22c08b24ba01ac4524d77bcb1da,addb88a34374d43aa9ecd4df7359ce39,NaN,NaN,NaN,False,False,False,‎Acesse este link para entrar no meu grupo do ...,...,2022-10-10 23:36:19.724987,-0.1531,NaN,439A91ADD8F355CD23C4BB107A5E88BB,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,acesse entrar grupo whatsapp chat
7,2022-10-10 16:49:48,325720ed3339a91b8076df12c1e95c45,0e345813dcb62b0fe4d8537f311af0f1,NaN,NaN,https://m.kwai.com/photo/150000006567403/52240...,False,True,False,Fortes palavras da Pastora Damares!\nhttps://k...,...,2022-10-10 16:49:50.051126,0.0000,NaN,F18BADED5AFA8FB0C33FE36625872DB8,558594228826.0:12@s.whatsapp.net,TextoExtendido,whatsapp,NaN,NaN,fortes palavras pastora damares
8,2022-10-10 16:49:53,8ed44a70a011285622e2b8919c2c8c3e,3b3e64e81c3e3f3bfc5ba76ddb90fb2d,0d6f2896e5941ef1933e69bbd7a32f69.jpeg,image/jpeg,NaN,True,False,False,🤣🤣🤣🤣🤣🤣🤣🤣,...,2022-10-10 16:49:54.713831,0.0000,NaN,724C233591C3A62A5231B47FF22ADE4B,558594228826.0:12@s.whatsapp.net,Imagem,whatsapp,NaN,0d6f2896e5941ef1933e69bbd7a32f69,


## Modelagem de tópicos sem filtros

In [8]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_geral['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Downloading: 1.02kB [00:00, ?B/s]
Downloading: 100%|██████████| 190/190 [00:00<?, ?B/s] 
Downloading: 3.89kB [00:00, ?B/s]
Downloading: 100%|██████████| 122/122 [00:00<?, ?B/s] 
Downloading: 100%|██████████| 471M/471M [01:49<00:00, 4.32MB/s] 
Downloading: 100%|██████████| 470M/470M [01:52<00:00, 4.20MB/s] 
Downloading: 100%|██████████| 470M/470M [02:07<00:00, 3.70MB/s] 
Downloading: 100%|██████████| 470M/470M [01:50<00:00, 4.26MB/s] 
Downloading: 100%|██████████| 470M/470M [01:51<00:00, 4.21MB/s] 
Downloading: 100%|██████████| 235M/235M [00:48<00:00, 4.87MB/s] 
Downloading: 100%|██████████| 118M/118M [00:25<00:00, 4.69MB/s] 
Downloading: 100%|██████████| 118M/118M [00:24<00:00, 4.89MB/s] 
Downloading: 100%|██████████| 118M/118M [00:25<00:00, 4.72MB/s] 
Downloading: 100%|██████████| 118M/118M [00:25<00:00, 4.68MB/s] 
Downloading: 100%|██████████| 470M/470M [01:39<00:00, 4.74MB/s] 
Downloading: 399kB [00:00, 199MB/s]
Downloading: 100%|██████████| 119M/119M [00:24<00:00, 4.92MB/s] 
Downlo

In [9]:
# Exibir as top 10 palavras de um tópico específico
topic_model.get_topic(0)

# Exibir os 10 principais tópicos
for i in range(10):
    print(f"Tópico {i}:", topic_model.get_topic(i))

# palavra, valor 
# valor = Importância (ou Relevância) da Palavra dentro do Tópico

Tópico 0: [('at', 4.47337641326211), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Tópico 1: [('votos', 0.010756959316334512), ('votar', 0.010418653189363974), ('eleitores', 0.007789550696661582), ('validos', 0.007700333775224411), ('voto', 0.007561613144114401), ('urnas', 0.006929964866323706), ('vota', 0.0066089787905531435), ('votou', 0.005326577738432256), ('votacao', 0.004989989480312604), ('pesquisa', 0.004532241247003797)]
Tópico 2: [('facebook', 0.7442485134834548), ('ipva', 0.0013750052792330608), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Tópico 3: [('twitter', 0.7618980153707896), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05), ('', 1e-05)]
Tópico 4: [('paulista', 0.00663626003696214), ('poderaopovo', 0.005306388269537009), ('atrombetanews', 0.0048006682137990125), ('amigo', 0.0045687993983

In [10]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/wpp_bertopic_sem_filtro.csv', index=False)

In [11]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Melissa Felipe\.conda\envs\topics_env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
# Diversidade de tópicos
def topic_diversity(topics_words, top_n=10):
    words = [word for topic in topics_words for word in topic[:top_n]]
    unique_words = set(words)
    return len(unique_words) / (len(topics_words) * top_n)


# IRBO

def irbo(topics, topk=10, p=0.9):
    
    def rbo(list1, list2, p):
        overlap = 0.0
        rbo_score = 0.0
        depth = min(len(list1), len(list2))
        for d in range(1, depth + 1):
            if list1[d-1] in list2[:d] and list2[d-1] in list1[:d]:
                overlap += 1
            rbo_score += overlap / d * (p ** d)
        return (1 - p) * rbo_score

    scores = []
    for i in range(len(topics)):
        for j in range(i + 1, len(topics)):
            t1 = topics[i][:topk]
            t2 = topics[j][:topk]
            score = rbo(t1, t2, p)
            scores.append(score)

    return 1 - np.mean(scores) if scores else 0.0


In [13]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.5082
Coerência c_npmi:  nan
Coerência c_uci:   nan
Coerência u_mass:  nan
Diversidade:       0.6991
IRBO:              0.9998


## Com filtro religioso

In [14]:
palavras_religiosas = [
    "deus", "jesus", "misericordia", "davi",
    "salomao", "reino", "templo", "conservador",
    "pentecostal", "rcc", "renovacao", "carismatic",
    "paulo ricardo", "bernardo kuster", "herege", "ateu",
    "jerico", "heresia"
]


In [15]:
# Função para verificar se uma palavra está relacionada à religião
def relacionada_religiao(word):
    word_lower = word.lower()
    palavras_religiosas_lower = [palavra.lower() for palavra in palavras_religiosas]

    # Verificando se alguma palavra da lista de palavras religiosas está presente
    return any(palavra in word_lower for palavra in palavras_religiosas_lower)


df_religiao = df_geral[df_geral['text_processed'].apply(lambda x: relacionada_religiao(x))]


In [16]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_religiao['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Batches: 100%|██████████| 485/485 [05:09<00:00,  1.57it/s]
2025-07-12 23:53:50,141 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-12 23:54:13,443 - BERTopic - Dimensionality - Completed ✓
2025-07-12 23:54:13,443 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-12 23:54:14,982 - BERTopic - Cluster - Completed ✓
2025-07-12 23:54:14,985 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-07-12 23:54:18,881 - BERTopic - Representation - Completed ✓


In [17]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/wpp_bertopic_filtro_religioso.csv', index=False)

In [18]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


In [19]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.6981
Coerência c_npmi:  0.0325
Coerência c_uci:   -2.9878
Coerência u_mass:  -2.1851
Diversidade:       0.7691
IRBO:              0.9972


## Com filtro e religioso e exclusão de politico

In [20]:
palavras_politicas = [ "lula", "bolsonaro", "pt", "pl", "stf", "patria", "55", "22", "13", "senadores", "lulaladrao",
                       "urnas", "alexandre", "moraes", "comunismo", "eleicao", "eleicoes", "esquerda", "direita",
                         "presidente", "tse", "fraude", "voto", "turno", "ministro"]

In [21]:
def retirar_mensagens_com_palavras_politicas(word):
    word_lower = word.lower()
    palavras_politica_lower = [palavra.lower() for palavra in palavras_politicas]

    return any(palavra in word_lower for palavra in palavras_politica_lower)


df_politico = df_religiao[~df_religiao['text_processed'].apply(lambda x: retirar_mensagens_com_palavras_politicas(x))]

In [22]:
# Treinando BERTopic com UMAP + HDBSCAN internos para clusterização

# Geração de Embeddings Semânticos com BERT
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Redução de Dimensionalidade com UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)

# Clustering com HDBSCAN
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=30, metric='euclidean', prediction_data=True)

# Modelagem de Tópicos com BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="portuguese",
    verbose=True
)


docs = df_politico['text_processed'].tolist()


embeddings = embedding_model.encode(docs, show_progress_bar=True)  

# Treinamento do modelo com embeddings já calculados
topics, probs = topic_model.fit_transform(docs, embeddings)



Batches: 100%|██████████| 162/162 [00:53<00:00,  3.01it/s]
2025-07-12 23:56:34,431 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-12 23:56:47,126 - BERTopic - Dimensionality - Completed ✓
2025-07-12 23:56:47,128 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-12 23:56:47,477 - BERTopic - Cluster - Completed ✓
2025-07-12 23:56:47,482 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-07-12 23:56:47,758 - BERTopic - Representation - Completed ✓


In [23]:
def get_topics_dataframe(topic_model, top_n):
    topics_data = []
    
    # Ignora o tópico -1 (outliers)
    for topic_id in topic_model.get_topics().keys():
        if topic_id == -1:
            continue
        
        # Lista de tuplas (palavra, peso)
        words_probs = topic_model.get_topic(topic_id)[:top_n]
        
        palavras = [word for word, _ in words_probs]
        pesos = [round(weight, 4) for _, weight in words_probs]

        topics_data.append({
            'Tópico': topic_id,
            'Palavras': ', '.join(palavras),
            'Pesos': pesos
        })

    return pd.DataFrame(topics_data).sort_values(by='Tópico').reset_index(drop=True)

df_topicos = get_topics_dataframe(topic_model, 50)

df_topicos.to_csv('./resultados_bertopic/wpp_bertopic_filtro_politico.csv', index=False)

In [24]:

# Extrair tópicos (lista de palavras) do BERTopic
topics_words = []

for topic_id in topic_model.get_topics().keys():
    if topic_id == -1:
        continue  # Ignora o outlier

    topic = topic_model.get_topic(topic_id)
    
    # Confere se é uma lista não vazia
    if topic and isinstance(topic, list):
        words = [word for word, _ in topic if isinstance(word, str) and word.strip()]
        if len(words) > 0:
            topics_words.append(words)


# Preparar textos tokenizados
texts = [doc.split() for doc in docs]

# Criar dicionário e corpus BoW
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

# Coerência
coh_cv = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_v').get_coherence()
coh_npmi = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_npmi').get_coherence()
coh_uci = CoherenceModel(topics=topics_words, texts=texts, dictionary=id2word, coherence='c_uci').get_coherence()
coh_umass = CoherenceModel(topics=topics_words, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()


In [25]:
div = topic_diversity(topics_words, top_n=10)
irbo_score = irbo(topics_words, topk=10)

# 7. Mostrar resultados
print("\n📊 Avaliação dos Tópicos:")
print(f"Coerência c_v:     {coh_cv:.4f}")
print(f"Coerência c_npmi:  {coh_npmi:.4f}")
print(f"Coerência c_uci:   {coh_uci:.4f}")
print(f"Coerência u_mass:  {coh_umass:.4f}")
print(f"Diversidade:       {div:.4f}")
print(f"IRBO:              {irbo_score:.4f}")



📊 Avaliação dos Tópicos:
Coerência c_v:     0.6595
Coerência c_npmi:  0.2318
Coerência c_uci:   0.2456
Coerência u_mass:  -3.1743
Diversidade:       0.8400
IRBO:              0.9801


In [26]:
df_topicos.head(10)

,Tópico,Palavras,Pesos
0,0,"deus, jesus, senhor, todos, nome, vida, vamos,...","[0.0777, 0.0372, 0.0352, 0.0227, 0.0213, 0.021..."
1,1,"brasil, deus, acima, todos, vamos, povo, senho...","[0.1217, 0.0598, 0.0546, 0.049, 0.0354, 0.0348..."
2,2,"cenas, fotos, melhor, 100, indecisos, violenci...","[0.1498, 0.149, 0.1475, 0.1469, 0.1469, 0.1462..."
3,3,"voltar, jesus, noticia, falando, sobre, redor,...","[1.955, 0.9749, 0.268, 0.2058, 0.1456, 0.1273,..."
4,4,"nunes, deixala, famosa, hostilio, 673, resende...","[0.5704, 0.3318, 0.3308, 0.3249, 0.3249, 0.324..."
